In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

sendgrid_api_key =os.environ.get("SENDGRID_API_KEY")
if sendgrid_api_key:
    print("Available")
else:
    print("No sendgrid Found")

Available


In [3]:
def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=sendgrid_api_key)
    from_email = Email("heartdan66@gmail.com")
    to_email = To("heartdan66@gmail.com")
    content = Content("text/plain","Hello Dan, this is text email")
    mail = Mail(from_email,to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)
send_test_email()

202


In [4]:
instruction1 =" You are part of an IEEE Rising working as Sponsorship Volunteer \
    IEEE Rising Stars is annual Conference that happens in Las Vegas in First Week of January Every Year\
        You write Professional, serious cold emails to companies and universities requesting to be part of the sponsors \
            The conference will take place next year 2026 on January 3-5, 2025 at Luxor Hotel"
    

instruction2 = "You are convincing agent part of an IEEE Rising working as Sponsorship Volunteer \
    IEEE Rising Stars is annual Conference that happens in Las Vegas in First Week of January Every Year\
        You write witty, enging cold emails to universities andc companies requesting to be part of the sponsors\
            that are likely to get a response\
                The conference will take place next year 2026 on January 3-5, 2025 at Luxor Hotel"

instruction3 = "You are busy part of an IEEE Rising working as Sponsorship Volunteer \
    IEEE Rising Stars is annual Conference that happens in Las Vegas in First Week of January Every Year\
        You conncise, and to the point cold emails universities andc companies requesting to be part of the sponsors\
            that are likely to get a response\
                The conference will take place next year 2026 on January 3-5, 2025 at Luxor Hotel"     

In [5]:
volunteer_agent1= Agent(
    name = " Professional Volunteer",
    instructions = instruction1,
    model = "gpt-4o-mini"
)

volunteer_agent2=Agent(
    name="Convicing Agent",
    instructions = instruction2,
    model="gpt-4o-mini"


)
volunteer_agent3=Agent(
    name="Busy_Volunteer",
    instructions = instruction3,
    model="gpt-4o-mini"
)

In [6]:
with trace("Sponsor Agent"):
    result = await Runner.run(volunteer_agent1, "Write a Cold Email")
    print(result.final_output)

Subject: Sponsorship Opportunity at IEEE Rising Stars Conference 2026

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am a Sponsorship Volunteer for the IEEE Rising Stars Conference, an annual event that brings together emerging professionals in the engineering and technology fields. We are excited to announce that the next conference will take place from January 3-5, 2026, at the Luxor Hotel in Las Vegas.

The IEEE Rising Stars Conference provides a unique platform for networking, knowledge sharing, and collaboration among engineers, academic professionals, and industry leaders. As a sponsor, your organization will have the opportunity to position itself as a thought leader in the technology space, engage with a diverse audience, and showcase your commitment to fostering the next generation of talent in our industry.

We are currently seeking sponsors who are aligned with our mission to inspire and empower young engineers and technologists.

In [7]:
result = Runner.run_streamed(volunteer_agent1, input="Write a cool sales Email")
async for event in result.stream_events():
    if event.type =="raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end ="", flush =True)

Subject: Partnership Opportunity: Showcase Your Brand at IEEE Rising Stars 2026

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am reaching out on behalf of the IEEE Rising Stars Conference, an annual event that gathers bright minds in the engineering and technology sectors. This premier conference will be held from January 3-5, 2026, at the Luxor Hotel in Las Vegas.

As a dynamic platform that fosters innovation and collaboration, IEEE Rising Stars attracts a diverse audience, including students, professionals, and industry leaders keen on advancing their knowledge and networks. With your company's commitment to [relevant field/industry], we believe a partnership could offer significant value.

**Why Sponsor IEEE Rising Stars 2026?**
- **Brand Exposure:** Gain visibility among a targeted audience of over [number] attendees, including future talent and industry influencers.
- **Networking Opportunities:** Connect with potential collaborators

In [8]:
message = "Writea cold sponsor email"

with trace("Parallel Cold Emails"):
    results = await asyncio.gather(
        Runner.run(volunteer_agent1, message),
        Runner.run(volunteer_agent2, message),
        Runner.run(volunteer_agent3, message)
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output +"\n\n")

Subject: Partnership Opportunity at IEEE Rising Stars Conference 2026

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am a Sponsorship Volunteer for the IEEE Rising Stars Conference, which will take place from January 3-5, 2026, at the Luxor Hotel in Las Vegas. 

The IEEE Rising Stars Conference serves as a premier platform for networking and collaboration among emerging leaders, innovators, and professionals in the field of engineering and technology. With an expected attendance of over [insert expected number] students, early career professionals, and industry leaders, this event presents a unique opportunity for organizations like yours to connect with the brightest minds and showcase your commitment to advancing technology and innovation.

We are currently seeking sponsors who share our vision of fostering learning and professional development within the engineering community. By partnering with us, your organization will gain visibility

In [9]:
volunteer_pick = Agent(
    name="Volunteer_Picker",
    instructions=" You pick the best cold emails from given options. \
        Imagine you are a sponosr and pick the one you are most likely to response to. \
            Dont not give an explanation; reply with the selected email only. ",
    model ="gpt-4o-mini"
)

In [10]:
message = "Write a cold Volunteer Email"

with trace("Selection from Volunteers"):
    result = await asyncio.gather(
        Runner.run(volunteer_agent1, message),
        Runner.run(volunteer_agent2, message),
        Runner.run(volunteer_agent3, message)

    )

outputs = [result.final_output for result in results]

emails ="Cold Sponsor Emails:\n\n" + "\n\nEmail".join(outputs)

best = await Runner.run(volunteer_pick, emails)

print(f"Best Sponsor Email:\n{best.final_output}")

Best Sponsor Email:
Subject: Elevate Your Brand at the IEEE Rising Stars Conference 2026! 

Dear [Recipient's Name],

I hope this email finds you well and thriving! As we dive into the future of technology and innovation, I’m reaching out on behalf of the IEEE Rising Stars Conference happening from January 3-5, 2026, at the vibrant Luxor Hotel in Las Vegas.

Imagine this: over a thousand enthusiastic minds coming together to connect, collaborate, and revolutionize the future of engineering. This is your chance to be front and center as a sponsor, positioning your brand among the brightest talent in the industry!

**Why Sponsor?**  
- **Brand Visibility**: Showcase your company to future engineers, industry leaders, and potential partners.
- **Networking Opportunities**: Engage with eager students and professionals who are looking for real-world connections and opportunities.
- **Thought Leadership**: Share your insights through keynote sessions or workshops, establishing your organizat

## Using tools 

In [11]:
volunteer_agent1 = Agent(
    name="Professional Volunteer",
    instructions=instruction1,
    model="gpt-4o-mini"
)

volunteer_agent2 = Agent(
    name="Convicng Agent",
    instructions=instruction2,
    model ="gpt-4o-mini"

)

volunteer_agent3 = Agent(
    name="Busy Volunteer",
    instructions=instruction3,
    model ="gpt-4o-mini"
)

In [12]:
volunteer_agent1

Agent(name='Professional Volunteer', instructions=' You are part of an IEEE Rising working as Sponsorship Volunteer     IEEE Rising Stars is annual Conference that happens in Las Vegas in First Week of January Every Year        You write Professional, serious cold emails to companies and universities requesting to be part of the sponsors             The conference will take place next year 2026 on January 3-5, 2025 at Luxor Hotel', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True

In [13]:
@function_tool
def send_email(body:str):

    sg = sendgrid.SendGridAPIClient(api_key=sendgrid_api_key)
    from_email = Email("heartdan66@gmail.com")
    to_email = To("dahimbisibwe66@gmail.com")
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sponsor Email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status":"success"}

In [14]:
send_email

FunctionTool(name='send_email', description='', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000029B81672520>, strict_json_schema=True, is_enabled=True)

In [15]:
tool1 = volunteer_agent1.as_tool(tool_name="volunteer_agent1", tool_description="Write a cold sponsor email")
tool1

FunctionTool(name='volunteer_agent1', description='Write a cold sponsor email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'volunteer_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000029B816737E0>, strict_json_schema=True, is_enabled=True)

In [16]:

description = "Write a cold Sponsor Email"

tool1 = volunteer_agent1.as_tool(tool_name="volunteer_agent1", tool_description=description)
tool2 = volunteer_agent2.as_tool(tool_name="volunteer_agent2", tool_description=description)
tool3 = volunteer_agent3.as_tool(tool_name='volunteer_agent3', tool_description=description)

In [17]:
tools = [tool1, tool2, tool3]

In [18]:
tools

[FunctionTool(name='volunteer_agent1', description='Write a cold Sponsor Email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'volunteer_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000029B81673A60>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='volunteer_agent2', description='Write a cold Sponsor Email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'volunteer_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000029B816739C0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='volunteer_agent3', description='Write a cold Sponsor Email', params_json_schema={'properties': {'input': {'title':

In [19]:
instructions = """

You are acting as Chair of the IEEE Rising Stars Sponsorshop Committtee. Your goal is to find the best cold emails using the volunteer_agents
Follow these steps carefully:
1. Generate Drafts: use all these 3 tools to genreate 3 diffetmet email drafts. Dont proceed until all 3 drafts are ready
2. use the send_email tool to send the best email (and only the best email) to the user

crucial rules:
You must use the agents to generate the drafts
You must sned ONE Email using the send email too - never more than open

"""

In [20]:
# volunteer_manager = Agent(name="Volunteer_Manager", instructions=instructions, tools = tools, model="gpt-4o-mini")

# message = "Send a cold sponsor email addressed as Dear Potential Sponsor"

# with trace("Volunter Manager"):
#     result = await Runner.run(
#         volunteer_manager,
#         message
#     )